In [66]:
import pandas as pd
import numpy as np
import json

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Ресурсы
https://towardsdatascience.com/three-cases-when-parsing-json-documents-in-a-python-pandas-dataframe-9c9dfea9ec34

## Как переформатировать bytes-json-строку в многоуровневый датафрейм?

In [129]:
bytes_string = '\n'.join([
    '{"col1": "a", "col2": 1, "char": {"group": "A", "source": "omega", "coordinates": {"latitude": "83C", "longitude": "50C"}}, "agent": {"name": "GP", "spec": "doctor"}}', 
    '{"col1": "b", "col2": 2, "char": {"group": "B", "source": "beta", "coordinates": {"latitude": "63C", "longitude": "70C"}}, "agent": {"name": "Fury", "spec": "Curator"}}', 
    '{"col1": "g", "col2": 3, "char": {"group": "G", "source": "gamma", "coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}',
    '{"col1": "c", "col2": 3, "char": {"coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}',
    '{"col1": "c", "col2": 3, "char": {"source": "cetta", "coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}',
    '{"col1": "c", "col2": 3, "char": {"coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}',
    '{"col1": "c", "col2": 3, "char": {"coordinates": {"wrong_key": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}'
    
]).encode('utf-8')

bytes_string

b'{"col1": "a", "col2": 1, "char": {"group": "A", "source": "omega", "coordinates": {"latitude": "83C", "longitude": "50C"}}, "agent": {"name": "GP", "spec": "doctor"}}\n{"col1": "b", "col2": 2, "char": {"group": "B", "source": "beta", "coordinates": {"latitude": "63C", "longitude": "70C"}}, "agent": {"name": "Fury", "spec": "Curator"}}\n{"col1": "g", "col2": 3, "char": {"group": "G", "source": "gamma", "coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}\n{"col1": "c", "col2": 3, "char": {"coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}\n{"col1": "c", "col2": 3, "char": {"source": "cetta", "coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}\n{"col1": "c", "col2": 3, "char": {"coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}\n{"col1": "c", "col2": 3, "char": {"coordinat

In [130]:
string = bytes_string.decode('utf-8')
string

'{"col1": "a", "col2": 1, "char": {"group": "A", "source": "omega", "coordinates": {"latitude": "83C", "longitude": "50C"}}, "agent": {"name": "GP", "spec": "doctor"}}\n{"col1": "b", "col2": 2, "char": {"group": "B", "source": "beta", "coordinates": {"latitude": "63C", "longitude": "70C"}}, "agent": {"name": "Fury", "spec": "Curator"}}\n{"col1": "g", "col2": 3, "char": {"group": "G", "source": "gamma", "coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}\n{"col1": "c", "col2": 3, "char": {"coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}\n{"col1": "c", "col2": 3, "char": {"source": "cetta", "coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}\n{"col1": "c", "col2": 3, "char": {"coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}\n{"col1": "c", "col2": 3, "char": {"coordinate

In [131]:
jsons_list = string.splitlines()
jsons_list

['{"col1": "a", "col2": 1, "char": {"group": "A", "source": "omega", "coordinates": {"latitude": "83C", "longitude": "50C"}}, "agent": {"name": "GP", "spec": "doctor"}}',
 '{"col1": "b", "col2": 2, "char": {"group": "B", "source": "beta", "coordinates": {"latitude": "63C", "longitude": "70C"}}, "agent": {"name": "Fury", "spec": "Curator"}}',
 '{"col1": "g", "col2": 3, "char": {"group": "G", "source": "gamma", "coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}',
 '{"col1": "c", "col2": 3, "char": {"coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}',
 '{"col1": "c", "col2": 3, "char": {"source": "cetta", "coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}',
 '{"col1": "c", "col2": 3, "char": {"coordinates": {"latitude": "73C", "longitude": "30C"}}, "agent": {"name": "Boston", "spec": "Assassin"}}',
 '{"col1": "c", "col2": 3, "

In [132]:
def pandas_normalizer(df, STOP_WORDS = [], SEP='.'):
    dict_cols = [col for col in df.columns if dict in df[col].dropna().apply(type).unique()]
    filtered_dict_cols = [col for col in dict_cols if not any([pat in col for pat in STOP_WORDS])]
    
    for col in filtered_dict_cols:
        add = pd.json_normalize(df[col], sep = SEP)
        add.columns = [col + SEP + element for element in add]
        df[add.columns] = add
        df = df.drop(col, axis=1)
        
    return df
    

def recursive_unpacker(df, STOP_WORDS = [], SEP='.'):
    dict_cols = [col for col in df.columns if dict in df[col].dropna().apply(type).unique()]
    filtered_dict_cols = [col for col in dict_cols if not any([pat in col for pat in STOP_WORDS])]
    
    for col in filtered_dict_cols:
        addit_df = df[col].apply(pd.Series)
        addit_df.columns = [SEP.join([col, add]) for add in addit_df.columns]
        addit_df = recursive_unpacker(addit_df, STOP_WORDS=STOP_WORDS, SEP=SEP)

        df = pd.concat([df, addit_df], axis=1)
        df = df.drop(col, axis=1)

    return df

simple_dfs_list = []
rec_dfs_list = []
norm_dfs_list = []

for json_string in jsons_list:
    json_val = json.loads(json_string)
    
    simple_df = pd.DataFrame.from_dict(json_val, orient='index').T
    simple_dfs_list.append(simple_df)
    
    df = simple_df.copy()
    norm_df = pandas_normalizer(df, STOP_WORDS=['coordinates'], SEP='__')
    norm_dfs_list.append(norm_df)
    
    df = simple_df.copy()
    rec_df = recursive_unpacker(df, STOP_WORDS=['coordinates'], SEP='__')
    rec_dfs_list.append(rec_df)
    
simple_data = pd.concat(simple_dfs_list)
norm_data = pd.concat(norm_dfs_list)
rec_data = pd.concat(rec_dfs_list)

In [133]:
simple_data

,col1,col2,char,agent
0,a,1,"{'group': 'A', 'source': 'omega', 'coordinates...","{'name': 'GP', 'spec': 'doctor'}"
0,b,2,"{'group': 'B', 'source': 'beta', 'coordinates'...","{'name': 'Fury', 'spec': 'Curator'}"
0,g,3,"{'group': 'G', 'source': 'gamma', 'coordinates...","{'name': 'Boston', 'spec': 'Assassin'}"
0,c,3,"{'coordinates': {'latitude': '73C', 'longitude...","{'name': 'Boston', 'spec': 'Assassin'}"
0,c,3,"{'source': 'cetta', 'coordinates': {'latitude'...","{'name': 'Boston', 'spec': 'Assassin'}"
0,c,3,"{'coordinates': {'latitude': '73C', 'longitude...","{'name': 'Boston', 'spec': 'Assassin'}"
0,c,3,"{'coordinates': {'wrong_key': '73C', 'longitud...","{'name': 'Boston', 'spec': 'Assassin'}"


In [134]:
norm_data

,col1,col2,char__group,char__source,char____coordinates__latitude,char____coordinates__longitude,agent__name,agent__spec,char____coordinates__wrong_key
0,a,1,A,omega,83C,50C,GP,doctor,NaN
0,b,2,B,beta,63C,70C,Fury,Curator,NaN
0,g,3,G,gamma,73C,30C,Boston,Assassin,NaN
0,c,3,NaN,NaN,73C,30C,Boston,Assassin,NaN
0,c,3,NaN,cetta,73C,30C,Boston,Assassin,NaN
0,c,3,NaN,NaN,73C,30C,Boston,Assassin,NaN
0,c,3,NaN,NaN,NaN,30C,Boston,Assassin,73C


In [135]:
rec_data

,col1,col2,char__group,char__source,char__coordinates,agent__name,agent__spec
0,a,1,A,omega,"{'latitude': '83C', 'longitude': '50C'}",GP,doctor
0,b,2,B,beta,"{'latitude': '63C', 'longitude': '70C'}",Fury,Curator
0,g,3,G,gamma,"{'latitude': '73C', 'longitude': '30C'}",Boston,Assassin
0,c,3,NaN,NaN,"{'latitude': '73C', 'longitude': '30C'}",Boston,Assassin
0,c,3,NaN,cetta,"{'latitude': '73C', 'longitude': '30C'}",Boston,Assassin
0,c,3,NaN,NaN,"{'latitude': '73C', 'longitude': '30C'}",Boston,Assassin
0,c,3,NaN,NaN,"{'wrong_key': '73C', 'longitude': '30C'}",Boston,Assassin


## Как из нескольких неполных строк получить одну полную?

In [136]:
rec_data

,col1,col2,char__group,char__source,char__coordinates,agent__name,agent__spec
0,a,1,A,omega,"{'latitude': '83C', 'longitude': '50C'}",GP,doctor
0,b,2,B,beta,"{'latitude': '63C', 'longitude': '70C'}",Fury,Curator
0,g,3,G,gamma,"{'latitude': '73C', 'longitude': '30C'}",Boston,Assassin
0,c,3,NaN,NaN,"{'latitude': '73C', 'longitude': '30C'}",Boston,Assassin
0,c,3,NaN,cetta,"{'latitude': '73C', 'longitude': '30C'}",Boston,Assassin
0,c,3,NaN,NaN,"{'latitude': '73C', 'longitude': '30C'}",Boston,Assassin
0,c,3,NaN,NaN,"{'wrong_key': '73C', 'longitude': '30C'}",Boston,Assassin


In [163]:
rec_data_whole = (
    rec_data
        .groupby('col1', as_index=False)
        .apply(
            lambda group: print(group.apply(lambda col: col.dropna().drop_duplicates(), axis=0))
        )
        .reset_index(drop=True)
        .drop_duplicates(subset=['col1'])
)

rec_data_whole

  col1 col2 char__group char__source                        char__coordinates  \
0    a    1           A        omega  {'latitude': '83C', 'longitude': '50C'}   

  agent__name agent__spec  
0          GP      doctor  
  col1 col2 char__group char__source                        char__coordinates  \
0    b    2           B         beta  {'latitude': '63C', 'longitude': '70C'}   

  agent__name agent__spec  
0        Fury     Curator  
  col1 col2 char__group char__source  \
0    c    3         NaN        cetta   
0    c    3         NaN        cetta   

                          char__coordinates agent__name agent__spec  
0   {'latitude': '73C', 'longitude': '30C'}      Boston    Assassin  
0  {'wrong_key': '73C', 'longitude': '30C'}      Boston    Assassin  
  col1 col2 char__group char__source                        char__coordinates  \
0    g    3           G        gamma  {'latitude': '73C', 'longitude': '30C'}   

  agent__name agent__spec  
0      Boston    Assassin  


""


In [137]:
rec_data_whole = (
    rec_data
        .groupby('col1', as_index=False)
        .apply(
            lambda group: group.apply(lambda col: col.dropna().drop_duplicates(), axis=0)
        )
        .reset_index(drop=True)
        .drop_duplicates(subset=['col1'])
)

rec_data_whole

,col1,col2,char__group,char__source,char__coordinates,agent__name,agent__spec
0,a,1,A,omega,"{'latitude': '83C', 'longitude': '50C'}",GP,doctor
1,b,2,B,beta,"{'latitude': '63C', 'longitude': '70C'}",Fury,Curator
2,c,3,NaN,cetta,"{'latitude': '73C', 'longitude': '30C'}",Boston,Assassin
4,g,3,G,gamma,"{'latitude': '73C', 'longitude': '30C'}",Boston,Assassin


## Как распаковать колонку со списками?